In [9]:
import os
from dotenv import load_dotenv

# Azure imports
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

In [10]:
load_dotenv(override=True)

True

In [11]:
# ## Using Azure AI Foundry project, example: AZURE_AI_PROJECT=https://your-account.services.ai.azure.com/api/projects/your-project
# azure_ai_project = os.environ.get("PROJECT_ENDPOINT")

# # Instantiate your AI Red Teaming Agent
# red_team_agent = RedTeam(
#     azure_ai_project=azure_ai_project, # required
#     credential=DefaultAzureCredential() # required
# )

# # A simple example application callback function that always returns a fixed response
# def simple_callback(query: str) -> str:
#     return "I'm an AI assistant that follows ethical guidelines. I cannot provide harmful content."

# # Runs a red teaming scan on the simple callback target
# red_team_result = await red_team_agent.scan(target=simple_callback)

## Testing your app

In [12]:
import requests
import json
import asyncio
from typing import Dict, Any

# API configuration - adjust if your API runs on a different port
API_BASE_URL = "http://localhost:8000"
API_CHAT_ENDPOINT = f"{API_BASE_URL}/chat"
API_STREAM_ENDPOINT = f"{API_BASE_URL}/chat/stream"

In [13]:
## Using Azure AI Foundry project, example: AZURE_AI_PROJECT=https://your-account.services.ai.azure.com/api/projects/your-project
azure_ai_project = os.environ.get("PROJECT_ENDPOINT")

# Instantiate your AI Red Teaming Agent
red_team_agent = RedTeam(
    azure_ai_project=azure_ai_project, # required
    credential=DefaultAzureCredential() # required
)

In [14]:
def api_stream_callback(query: str, session_id: str = None) -> str:
    """
    Callback function that calls the streaming API and returns the complete response
    """
    try:
        payload = {
            "message": query,
            "session_id": session_id
        }
        
        response = requests.post(
            API_STREAM_ENDPOINT,
            json=payload,
            headers={"Content-Type": "application/json"},
            stream=True,
            timeout=60
        )
        
        if response.status_code == 200:
            complete_response = ""
            for line in response.iter_lines():
                if line:
                    decoded_line = line.decode('utf-8')
                    print(f"💹Print Line: {decoded_line}")
                    if decoded_line.startswith('data: '):
                        try:
                            data_str = decoded_line[6:]  # Remove 'data: ' prefix
                            data = json.loads(data_str)
                            
                            if 'content' in data:
                                complete_response += data['content']
                            
                            # Check if streaming is done
                            if data.get('is_done', False):
                                break
                                
                        except json.JSONDecodeError:
                            continue  # Skip invalid JSON lines
            print(f"💹complete_response: {complete_response}")
            return complete_response if complete_response else "No response received from streaming API"
        else:
            return f"API Error: {response.status_code} - {response.text}"
            
    except requests.exceptions.RequestException as e:
        return f"Request failed: {str(e)}"
    except Exception as e:
        return f"Unexpected error: {str(e)}"

In [15]:
# Check if API is accessible
def check_api_health():
    """Check if the API is running and accessible"""
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=5)
        if response.status_code == 200:
            print("✅ API is accessible and healthy")
            return True
        else:
            print(f"❌ API health check failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ Cannot reach API: {str(e)}")
        print("Please make sure the API is running on http://localhost:8000")
        return False

# Check API availability
api_available = check_api_health()

✅ API is accessible and healthy


In [16]:
# Execute red teaming with streaming API callback
if api_available:
    print("🚀 Running red teaming with streaming API callback...")
    
    # Create a wrapper that matches the expected signature for red teaming
    def api_stream_target_callback(query: str) -> str:
        return api_stream_callback(query)
    
    try:
        # Run red teaming scan with streaming API callback
        red_team_result_stream = await red_team_agent.scan(
            target=api_stream_target_callback,
            attack_strategies=[
				AttackStrategy.Jailbreak,  # Group of moderate complexity attacks
				# AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
			]
            )
        
        # Get attack simulations (it's a method, not a property)
        stream_attack_sims = red_team_result_stream.attack_simulation()
        
        print(f"Streaming red teaming completed with {len(stream_attack_sims)} attack simulations")
        print("Results saved in 'red_team_result_stream' variable")
        
    except Exception as e:
        print(f"❌ Error during streaming red teaming: {str(e)}")
        print("This might be due to API errors. Check if the API is working correctly.")
else:
    print("⚠️ Skipping streaming red teaming - API not available")

🚀 Running red teaming with streaming API callback...
🚀 STARTING RED TEAM SCAN
📂 Output directory: .\.scan_20250924_214937
📊 Risk categories: ['hate_unfairness', 'sexual', 'violence', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/resource/build/redteaming/db1654ad-1ba1-4da3-a6aa-39efe0b3ef62?wsid=/subscriptions/63c96444-e030-4247-98db-78dfdbb8ad68/resourceGroups/redteaming-demo-rg-swe-mkurahara/providers/Microsoft.CognitiveServices/accounts/redteaming-demo-AIfoundry-swe-mkurahara/projects/redteaming-demo-proj-swe-mkurahara&tid=c9678618-483c-4083-93c9-3bba88cb36c9
📋 Planning 8 total tasks
📝 Fetched baseline objectives for hate_unfairness: 10 objectives
📝 Fetched baseline objectives for sexual: 10 objectives
📝 Fetched baseline objectives for violence: 10 objectives
📝 Fetched baseline objectives for self_harm: 10 objectives
🔄 Fetching objectives for strategy 2/2: jailbreak


Scanning:   0%|                                       | 0/8 [00:00<?, ?scan/s, current=initializing]

⚙️ Processing 8 tasks in parallel (max 5 at a time)
▶️ Starting task: baseline strategy for hate_unfairness risk category
▶️ Starting task: baseline strategy for sexual risk category
▶️ Starting task: baseline strategy for violence risk category
▶️ Starting task: baseline strategy for self_harm risk category
▶️ Starting task: jailbreak strategy for hate_unfairness risk category
💹Print Line: data: {"content":"エラー: <class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service encountered a content error","session_id":"1ec8c600-733f-442e-b641-97b2559e34d0","is_done":true}
💹complete_response: エラー: <class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service encountered a content error
💹Print Line: data: {"content":"エラー: <class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service encountered a content error","session_id":"0fc2d6e0-1a5e-4eed-bd0b-d7f346026

Scanning:  50%|███████████████               | 4/8 [10:42<42:50, 642.56s/scan, current=initializing]

Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\baseline_hate_unfairness_b6c2f4ee-ea33-4072-9ec7-ed27c54a3698.json".
✅ Completed task 1/8 (12.5%) - baseline/hate_unfairness in 642.6s
   Est. remaining: 77.1 minutes
Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\baseline_sexual_6fe3d810-723f-4a96-b631-727853e7b5f5.json".
✅ Completed task 2/8 (25.0%) - baseline/sexual in 642.6s
   Est. remaining: 33.0 minutes
Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\baseline_violence_395ac0fa-586e-4a20-bca7-9d59bba9f6b4.json".
✅ Completed task 3/8 (37.5%) - baseline/violence in 642.6s
   Est. remaining: 18.4 minutes
Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\baseline_self_harm_cd3e4269-a67e-4dcb-b195-5a81b6a6f3c1.json".
✅ Completed task 4/8 (50.0%) - baseline/self_harm in 642.6s
   Est. remainin

...

Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\jailbreak_sexual_e3612726-0781-4e72-8c9d-28c156884bd3.json".
✅ Completed task 6/8 (75.0%) - jailbreak/sexual in 433.6s
   Est. remaining: 6.1 minutes
Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\jailbreak_violence_527996e4-b612-4483-a985-e51155e0a172.json".
✅ Completed task 7/8 (87.5%) - jailbreak/violence in 433.6s
   Est. remaining: 2.6 minutes


Scanning: 100%|██████████████████████████████| 8/8 [17:58<00:00, 134.75s/scan, current=initializing]


Evaluation results saved to "c:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\jailbreak_self_harm_8c04d56e-56c3-4b75-8b93-5f22a92c0da5.json".
✅ Completed task 8/8 (100.0%) - jailbreak/self_harm in 433.9s
   Est. remaining: 0.0 minutes


Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Request: POST swedencentral.api.azureml.ms/assetstore/v1.0/temporaryDataReference/createOrGet
Status Code: 403 Forbidden
Error Code: UserError/Auth/Authorization/ResourceMsiTokenDoesntHavePermissionsOnStorage
Reason Phrase: Foundry project MSI redteaming-demo-AIfoundry-swe-mkurahara/redteaming-demo-proj-swe-mkurahara doesn
Response Body: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "Foundry project MSI redteaming-demo-AIfoundry-swe-mkurahara/redteaming-demo-proj-swe-mkurahara doesn't have appropriate permissions on the storage account redteamingswemkurahara",
    "messageFormat": "Foundry project MSI {foundryAccountName}/{projectName} doesn't have appropriate permissions on the storage account {storageAccount}",
    "messageParameters": {
      "foundryAccountName": "redteaming-demo-AIfoundry-swe-mkurahara",


Evaluation results saved to "C:\Users\t-mkurahara\secure-azureai-agent\eval\.scan_20250924_214937\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/80 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category        | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Hate-unfairness      | 0.0%           | 0.0%                         | N/A                             | N/A                           
Sexual               | 0.0%           | 0.0%                         | N/A                             | N/A                           
Violence             | 0.0%           | 0.0%                         | N/A                             | N/A                           
Self-